In [1]:
! wget http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip

--2020-06-09 02:26:39--  http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip
Resolving model.scir.yunfutech.com (model.scir.yunfutech.com)... 47.89.64.233, 47.89.64.202, 47.89.64.204
Connecting to model.scir.yunfutech.com (model.scir.yunfutech.com)|47.89.64.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637993278 (608M) [application/zip]
Saving to: ‘ltp_data_v3.4.0.zip’

ltp_data_v3.4.0.zip 100%[===================>] 608.44M  98.2MB/s    in 6.6s    

2020-06-09 02:26:46 (92.6 MB/s) - ‘ltp_data_v3.4.0.zip’ saved [637993278/637993278]



In [2]:
!unzip ltp_data_v3.4.0.zip

Archive:  ltp_data_v3.4.0.zip
   creating: ltp_data_v3.4.0/
  inflating: ltp_data_v3.4.0/cws.model  
  inflating: ltp_data_v3.4.0/md5.txt  
  inflating: ltp_data_v3.4.0/ner.model  
  inflating: ltp_data_v3.4.0/parser.model  
  inflating: ltp_data_v3.4.0/pisrl.model  
  inflating: ltp_data_v3.4.0/pos.model  
  inflating: ltp_data_v3.4.0/version  


In [0]:
!pip install pyltp

     |████████████████████████████████| 5.3MB 27kB/s 


In [0]:
!tar -zxvf AutoMaster_TrainSet.tgz

In [0]:
!tar -zxvf ckpt.tgz

In [1]:
import tensorflow as tf
import numpy as np
from pyltp import SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
import pandas as pd
from gensim.models.word2vec import LineSentence, Word2Vec
import time, shutil, os
print(tf.__version__)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# shutil.rmtree('checkpoint')

2.2.0


In [4]:
vocabulary_dimension = 100
gru_unites = 128
attn_units = 32
batch_size = 128
learning_rate = 0.01
checkpoint_dir = './checkpoint'
EPOCHS = 15

In [5]:
df = pd.read_csv('./AutoMaster_TrainSet.csv')

In [6]:
cws_model = "./ltp_data_v3.4.0/cws.model"

def get_word_list(sentence=None,sentences=None,model=cws_model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    if sentences is not None:
        for i, s in enumerate(sentences):
            sentences[i] = list(segmentor.segment(s))
        return sentences
    else:
        word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

In [8]:
inp = list(df.Question)
outp = list(df.Report)
assert len(inp) == len(outp)

# 去除随时联系, 查阅资料等无意义回答
drop_index_set = set()
for index, (i, o) in enumerate(zip(inp, outp)):
    if not type(i) == type(o) == str or len(i) < 8 or len(o) < 8:
        drop_index_set.add(index)

inp = [el for i, el in enumerate(inp) if i not in drop_index_set]
outp = [el for i, el in enumerate(outp) if i not in drop_index_set]

inp = get_word_list(sentences=inp)
outp = get_word_list(sentences=outp)

In [9]:
def get_length(tensor):
    length = sorted([len(t) for t in tensor])
    return length[int(len(length) * 0.95)] + 2

inp_length = get_length(inp)
outp_length = get_length(outp)

with open('../gensim_input.txt', 'w') as f:
    for s in inp:
        s = ['<start>'] + s + ['<end>']
        f.write(' '.join(s) + '\n')
    for s in outp:
        s = ['<start>'] + s + ['<end>']
        f.write(' '.join(s) + '\n')

In [10]:
! head 10 ../gensim_input.txt

head: 10: No such file or directory
==> ../gensim_input.txt <==
<start> 2010 款 宝马 X1 ， 2011年 出厂 ， 2.0 排量 ， 通用 6L45 变速箱 ， 原地 换 挡 位 PRND 车辆 闯动 ， 行驶 升降档 正常 ， 4 轮 离 地 换 挡 无 冲击感 ， 更换 变速箱 油 12L 无 改变 。 试过 一 辆 2014年 进口 X1 原地 换 挡位 也 有 冲击感 ， 这 是 什么 情况 ， 哪里 的 问题 <end>
<start> 3.0V6 发动机 号 在 什么 位置 ， 有 照片 最 好 ！ <end>
<start> 2012款 奔驰 c180 怎么样 ， 维修 保养 ， 动力 ， 值得 拥有 吗 <end>
<start> 科鲁兹 变速箱 旁边 漏 机油 <end>
<start> 我 要 怎么 才 能 知道 车子 断开 电瓶 电源 之后 要 不 要 做 节气门 或 防盗 ， 音响 等 的 重置 ， 还有 节气门 重 置 是 不 是 所有 车型 都 一样 ， 打开 电源 不 启动 车子 ， 油门 踩 到底 五 秒 ， 重复 几 次 就 可以 了 。 其他 设备 要 密码 才 行 是 不 是 <end>
<start> 昌河 Q35 音响 怎么 拆 装 <end>
<start> 长安 35 朝阳 轮胎 不 要 里面 的 钢圈 。 用 我 自己 的 钢圈 多少 钱 外面 换 多少 钱 <end>
<start> 吉利 远景 外 球笼 上 那个 大 螺丝 是 顺时针 拧 下来 还是 反 的 <end>
<start> 五菱 之 光 6376nf ， 自己 装 了 铁将军 防盗器 ， 五线马达 的 白褐 两 根 信号线 接到 什么 地方 去 ？ 我 想 实现 一 控 三马达 <end>
<start> 丰田 花冠 行驶 了 十万 公里 皮带 要 换 吗 <end>


In [11]:
model = Word2Vec(corpus_file='../gensim_input.txt', size=vocabulary_dimension, min_count=10, workers=8, sg=0, negative=5, iter=5)
w2v_model = model

In [12]:
train_x_vocab = []
for s in inp:
    s = s[:inp_length - 2]
    s = ['<start>'] + s + ['<end>']
    s = s + ['<pad>'] * (inp_length - len(s))
    train_x_vocab.append(s)

train_y_vocab = []
for s in outp:
    s = s[:outp_length - 2]
    s = ['<start>'] + s + ['<end>']
    s = s + ['<pad>'] * (inp_length - len(s))
    train_y_vocab.append(s)

In [13]:
len(model.wv.vocab)

9045

In [14]:
model_vocab_size = len(model.wv.vocab)

inp_vocabulary_matrix = np.array([model.wv[model.wv.index2word[i]] for i in range(model_vocab_size)])

model.wv['<pad>'] = np.zeros((vocabulary_dimension,))
model.wv['<unknown>'] = np.average(inp_vocabulary_matrix, axis=0)

pad_wv_index = model.wv.vocab['<pad>'].index
unknown_wv_index = model.wv.vocab['<unknown>'].index

model_vocab_size = len(model.wv.vocab)
inp_vocabulary_matrix = [model.wv[model.wv.index2word[i]] for i in range(model_vocab_size)]

In [16]:
model.wv.index2word[9046]

'<unknown>'

In [17]:
train_x_token = []
train_x_extended_token = []
train_x_pad_mask = []
train_x_oov_token = []
train_x_oov_len = []
train_y_token = []
train_y_pad_mask = []

for s in train_x_vocab:
    s_word_index = []
    s_oov_index = []
    s_oov_dict = {}
    pad_mask = [1 for _ in range(len(s))]
    for i, w in enumerate(s):
        if w == '<unknown>':
            raise ValueError
        elif w == '<pad>':
            pad_mask[i] = 0
            s_word_index.append(pad_wv_index)
            s_oov_index.append(pad_wv_index)
        elif w in model.wv:
            s_word_index.append(model.wv.vocab[w].index)
            s_oov_index.append(model.wv.vocab[w].index)
        else:
            s_word_index.append(unknown_wv_index)
            if w not in s_oov_dict:
                s_oov_dict[w] = len(s_oov_dict)
            s_oov_index.append(model_vocab_size + s_oov_dict[w])
    
    train_x_token.append(s_word_index)
    train_x_extended_token.append(s_oov_index)
    train_x_pad_mask.append(pad_mask)
    train_x_oov_token.append(s_oov_dict)
    
train_x_oov_token = [list(el.keys()) for el in train_x_oov_token]
train_x_oov_len = [len(el) for el in train_x_oov_token]

for s in train_y_vocab:
    s_word_index = []
    pad_mask = [1 for _ in range(len(s))]
    for i, w in enumerate(s):
        if w == '<unknown>':
            raise ValueError
        elif w == '<pad>':
            pad_mask[i] = 0
            s_word_index.append(pad_wv_index)
        elif w in model.wv:
            s_word_index.append(model.wv.vocab[w].index)
        else:
            s_word_index.append(unknown_wv_index)
    train_y_token.append(s_word_index)
    train_y_pad_mask.append(pad_mask)

    

In [18]:
input_train, input_test, extended_input_train, extended_input_test, input_pad_mask_train, input_pad_mask_test, \
    input_oov_train_dict, input_oov_test_dict, input_oov_train_len, input_oov_test_len, output_train, output_test, \
    output_pad_mask_train, output_pad_mask_test = train_test_split(train_x_token, train_x_extended_token, train_x_pad_mask,
                                                                   train_x_oov_token, train_x_oov_len,train_y_token,train_y_pad_mask,
                                                                   test_size=0.05, random_state=6)
train_dataset_len = len(input_train)
test_dataset_len = len(input_test)

input_train, input_test, extended_input_train, extended_input_test, input_pad_mask_train, input_pad_mask_test, \
input_oov_train_len, input_oov_test_len, output_train, output_test, output_pad_mask_train, output_pad_mask_test = \
tf.convert_to_tensor(input_train), tf.convert_to_tensor(input_test), \
tf.convert_to_tensor(extended_input_train), tf.convert_to_tensor(extended_input_test), \
tf.convert_to_tensor(input_pad_mask_train), tf.convert_to_tensor(input_pad_mask_test), \
tf.convert_to_tensor(input_oov_train_len), tf.convert_to_tensor(input_oov_test_len), \
tf.convert_to_tensor(output_train), tf.convert_to_tensor(output_test), \
tf.convert_to_tensor(output_pad_mask_train), tf.convert_to_tensor(output_pad_mask_test)

print('train_test_split, train_input shape:', input_train.shape)
print('train_test_split, test_input shape:', input_test.shape)

dataset_train = tf.data.Dataset.from_tensor_slices(
(input_train, extended_input_train, input_pad_mask_train, input_oov_train_len, output_train, output_pad_mask_train))
dataset_test = tf.data.Dataset.from_tensor_slices(
(input_test, extended_input_test, input_pad_mask_test, input_oov_test_len, output_test, output_pad_mask_test))
dataset_train_batch = dataset_train.batch(batch_size=batch_size, drop_remainder=True)
dataset_test_batch = dataset_test.batch(batch_size=batch_size, drop_remainder=True)


train_test_split, train_input shape: (74728, 86)
train_test_split, test_input shape: (3934, 86)


In [19]:
dataset_train_batch, dataset_test_batch, dataset_train_len, dataset_test_len, dataset_train_oov_dict, \
    dataset_test_oov_dict = dataset_train_batch, dataset_test_batch, train_dataset_len, test_dataset_len, \
input_oov_train_dict, input_oov_test_dict

In [20]:
dataset_train_batch

<BatchDataset shapes: ((128, 86), (128, 86), (128, 86), (128,), (128, 86), (128, 86)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>

In [21]:
model = PGN(gru_unites, attn_units, batch_size, np.array(inp_vocabulary_matrix))

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05) # , clipvalue=2

In [24]:
checkpoint_prefix = os.path.join('./', "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=model.encoder, attention=model.attention,
                                 decoder=model.decoder, pointer=model.pointer)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [25]:
dataset_len = input_train.shape[0]
batch_sz = batch_size
steps_per_epoch = dataset_len // batch_sz

In [26]:
def train_one_batch(mode, w2v_model, model, optimizer, oov_dict, st, inp, targ, enc_extended_inp, enc_pad_mask,
                    batch_oov_len, cov_loss_wt, padding_mask=None):
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = model.call_encoder(inp)
        final_dist, attentions, coverages, _, _, _ = model(targ, enc_extended_inp, enc_pad_mask, batch_oov_len,
                                                enc_output, enc_hidden, use_coverage=True,prev_coverage=None)
        loss = loss_function(targ,final_dist,padding_mask)+cov_loss_wt*coverage_loss(attentions,coverages,padding_mask)
        batch_loss = (loss / int(targ.shape[1]))
        
        variables = model.encoder.trainable_variables + model.attention.trainable_variables + \
                        model.decoder.trainable_variables + model.pointer.trainable_variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        return batch_loss

In [27]:
save_chkp_epoch = 1
train_epoch = EPOCHS
cov_loss_wt = 0.5

In [28]:
mode, w2v_model, model, optimizer, checkpoint, checkpoint_prefix, save_chkp_epoch,\
dataset_batch, dataset_len, dataset_oov_dict, batch_sz, EPOCHS, cov_loss_wt = 'train', w2v_model, model, \
optimizer, checkpoint, checkpoint_prefix, save_chkp_epoch,\
dataset_train_batch, dataset_train_len, dataset_train_oov_dict, batch_sz, train_epoch, cov_loss_wt

In [29]:
for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    for (batch, (enc, enc_extend, enc_mask, enc_oov_len, dec, dec_mask)) in enumerate(dataset_train_batch.take(steps_per_epoch)):
        st = (batch + 1) * batch_sz
        raise
        batch_loss = train_one_batch('train',w2v_model,model,optimizer,dataset_oov_dict,st,enc,dec,enc_extend,enc_mask,
                                         enc_oov_len, cov_loss_wt=cov_loss_wt, padding_mask=dec_mask)
        total_loss += batch_loss
        if batch % 500 == 0:
                print('$$$$$$$$Epoch {} Batch {} Loss {:.4f}$$$$$$$$'.format(epoch, batch, batch_loss.numpy()))
    checkpoint.save(file_prefix=checkpoint_prefix)
    learning_rate *= 0.8
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=5)
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

RuntimeError: No active exception to reraise

In [ ]:
enc, enc_extend, enc_mask, enc_oov_len, dec, dec_mask

In [47]:
w2v_model.wv.index2word[9046]

'<unknown>'

In [52]:
enc[:10][:50]

<tf.Tensor: shape=(10, 86), dtype=int32, numpy=
array([[   2,   24,   55, 2397,  838,   37,  154, 3024,  100, 1839,   42,
         689,    4,   23,   31,    9,    3, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045],
       [   2, 2711,  611,  190,    0, 1369,   38,    0,  320,  286,  237,
        9046,  931,  142,    0,  132,  419,   72, 9046,  354,    0,  321,
        3069,    5,  320,   17,    5, 3544,   37,  488, 5135, 2676,    5,
         272, 1652,    6,    9,    3, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
 

In [53]:
enc_extend[:3]

<tf.Tensor: shape=(3, 86), dtype=int32, numpy=
array([[   2,   24,   55, 2397,  838,   37,  154, 3024,  100, 1839,   42,
         689,    4,   23,   31,    9,    3, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045],
       [   2, 2711,  611,  190,    0, 1369,   38,    0,  320,  286,  237,
        9047,  931,  142,    0,  132,  419,   72, 9048,  354,    0,  321,
        3069,    5,  320,   17,    5, 3544,   37,  488, 5135, 2676,    5,
         272, 1652,    6,    9,    3, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
  

In [54]:
enc_mask[:3]

<tf.Tensor: shape=(3, 86), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [35]:
enc_oov_len

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([0, 2, 3, 2, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 3, 0, 2, 0, 1, 2, 0, 1,
       0, 1, 0, 1, 1, 3, 4, 1, 3, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 1,
       0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 3, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 3, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 3, 2, 3, 0, 2, 0, 2, 1, 1, 0, 2, 0, 1, 0,
       0, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int32)>

In [57]:
dec[:3]

<tf.Tensor: shape=(3, 86), dtype=int32, numpy=
array([[   2,  154, 3024,    0, 2295, 3024,    0,  292,    1,  972,  135,
          49,    3, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045],
       [   2,  178,  305,   34,  283,    0,  192,   53, 3053,  187,    5,
           3, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
        9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045, 9045,
  

In [58]:
dec_mask[:3]

<tf.Tensor: shape=(3, 86), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [123]:
def get_shape(lis):
    return (len(lis), *lis[0].shape)

In [129]:
def loss_function(real, pred, padding_mask):
#     print(1, get_shape(real), get_shape(pred), get_shape(padding_mask))
#           1           (128, 86)        (85, 128, 9051)  (128, 86)

    #  pred is list of list, max_len_y * [batch_sz, extend_vocabsize] (without argmax) real & mask [batch_sz, max_len_y]
    #mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction="none")
    for t in range(real.shape[1]-1):
        
        loss_ = loss_object(real[:,t+1], pred[t])    # note this change 11.11 [batch_sz,]
        mask = tf.cast(padding_mask[:,t+1], dtype=loss_.dtype)
        loss_ *= mask
        # print('loss_:', loss_)
        loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
        loss += loss_
        # print('loss and loss at each time step(batch sum)', loss, loss_)
#     tf.print("loss:",loss)
    return loss


def coverage_loss(attn_dists, coverages, padding_mask):
#     print(2, get_shape(attn_dists), get_shape(coverages), get_shape(padding_mask))
#           2           (85, 128, 86, 1)       (85, 128, 86, 1)      (128, 86)
    # attn_dists: [max_len_y, batch_sz, max_len_x, 1]
    attn_dists = tf.squeeze(attn_dists, axis=-1)   # [max_len_y, batch_sz, max_len]
    coverage = tf.zeros_like(attn_dists[0])  # shape (batch_size, max_len_x). Initial coverage is zero.
    covlosses = []  # Coverage loss per decoder timestep. Will be list length max_dec_steps containing shape (batch_size).
    padding_mask = tf.stack(padding_mask, 1)   # [max_len_y, batch_sz]
    mask = tf.cast(padding_mask, dtype=attn_dists[1].dtype)
    for i,a in enumerate(attn_dists):
        covloss = tf.reduce_sum(tf.minimum(a, coverage), [1])  # calculate the coverage loss for this step
        # update the coverage vector
        coverage += a
        covloss = covloss * mask[i,:]
        covlosses.append(covloss)
    loss = tf.reduce_sum(tf.reduce_mean(covlosses, axis=0))
#     tf.print('coverage loss(batch sum):', loss)
    return loss

In [130]:
def train_one_batch(mode, w2v_model, model, optimizer, oov_dict, st, inp, targ, enc_extended_inp, enc_pad_mask,
                    batch_oov_len, cov_loss_wt, padding_mask=None):
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = model.call_encoder(inp)
        final_dist, attentions, coverages, _, _, _ = model(targ, enc_extended_inp, enc_pad_mask, batch_oov_len,
                                                enc_output, enc_hidden, use_coverage=True,prev_coverage=None)
        
#         print(final_dist[0].shape, attentions[0].shape, coverages[0].shape)
#         (128, 9051) (128, 86, 1) (128, 86, 1)
#         print(len(final_dist), len(attentions), len(coverages))
#         85 85 85
        
        loss = loss_function(targ,final_dist,padding_mask)+cov_loss_wt*coverage_loss(attentions,coverages,
                                                                                     padding_mask)
        raise
        
        batch_loss = (loss / int(targ.shape[1]))
        
        variables = model.encoder.trainable_variables + model.attention.trainable_variables + \
                        model.decoder.trainable_variables + model.pointer.trainable_variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        return batch_loss
train_one_batch('train',w2v_model,model,optimizer,dataset_oov_dict,st,enc,dec,enc_extend,enc_mask,
                                         enc_oov_len, cov_loss_wt=cov_loss_wt, padding_mask=dec_mask)

0 tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], shape=(128,), dtype=float32)
1 tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], shape=(128,), dtype=float32)
2 tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

29 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 0. 1. 0. 1.], shape=(128,), dtype=float32)
30 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 0. 1. 0. 1.], shape=(128,), dtype=float32)
31 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 

56 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
57 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
58 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

80 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
81 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
82 tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

RuntimeError: No active exception to reraise

In [2]:
class Encoder(tf.keras.layers.Layer):
    """
    calculate encoded output and hidden state from encoder input and initialized encoder hidden
    batch by batch and paragraph by paragraph
    input is [batch_sz,max_len_x] encoder hidden is [batch_sz, enc_units]
    output is [batch_sz, max_len_x, enc_units]
    """
    def __init__(self, enc_units, batch_sz, embedding_matrix):
        super(Encoder, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape[0], embedding_matrix.shape[1]
        self.batch_sz = batch_sz
        self.enc_units = enc_units // 2
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.bidirectional_gru = tf.keras.layers.Bidirectional(self.gru)

    def call(self, x, hidden):
        # [batch_sz,max_train_x,embedding_dim]
        x = self.embedding(x)
        # output is [batch_sz, max_train_x, enc_units] enc_hidden after concat is [batch_sz, enc_units]
        output, forward_state, backward_state = self.bidirectional_gru(x, initial_state=[hidden, hidden])
        enc_hidden = tf.keras.layers.concatenate([forward_state, backward_state],axis=-1)
        return output, enc_hidden

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))


class BahdanauAttention(tf.keras.layers.Layer):
    """
    calculate attention and coverage from dec_hidden enc_output and prev_coverage
    one dec_hidden(word) by one dec_hidden
    dec_hidden or query is [batch_sz, enc_unit], enc_output or values is [batch_sz, max_train_x, enc_units],
    prev_coverage is [batch_sz, max_len_x, 1]
    dec_hidden is initialized as enc_hidden, prev_coverage is initialized as None
    output context_vector [batch_sz, enc_units] attention_weights & coverage [batch_sz, max_len_x, 1]

    """
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_h = tf.keras.layers.Dense(units)
        self.W_s = tf.keras.layers.Dense(units)
        self.W_c = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_hidden, enc_output, enc_pad_mask, use_coverage, prev_coverage):

        # prev_coverage [batch_sz, max_len_x, 1]
        # 11.07 add enc_pad_mask [batch_sz, max_len_x] to mask attention
        # query or dec_hidden [batch_sz, enc_units], values or enc_output [batch_sz, max_len, enc_units]
        # hidden_with_time_axis [batch_size, 1, enc_units]
        hidden_with_time_axis = tf.expand_dims(dec_hidden, 1)

        if use_coverage and prev_coverage is not None:
            # self.W_s(values) [batch_sz, max_len, units] self.W_h(hidden_with_time_axis) [batch_sz, 1, units]
            # self.W_c(prev_coverage) [batch_sz, max_len, units]  score [batch_sz, max_len, 1]
            score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis) + self.W_c(prev_coverage)))
            # attention_weights shape (batch_size, max_len, 1)
            mask = tf.cast(enc_pad_mask, dtype=score.dtype)
            masked_score = tf.squeeze(score, axis=-1) * mask
            masked_score = tf.expand_dims(masked_score, axis=2)
            attention_weights = tf.nn.softmax(masked_score, axis=1)

            coverage = attention_weights + prev_coverage

        else:
            # self.W1(values) [batch_sz, max_len, units] self.W2(hidden_with_time_axis): [batch_sz, 1, units]
            # score [batch_sz, max_len, 1]
            score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis)))
            mask = tf.cast(enc_pad_mask, dtype=score.dtype)
            masked_score = tf.squeeze(score, axis=-1) * mask
            masked_score = tf.expand_dims(masked_score, axis=2)
            attention_weights = tf.nn.softmax(masked_score, axis=1)

            if use_coverage:
                coverage = attention_weights

        # [batch_sz, max_len, enc_units]
        context_vector = attention_weights * enc_output
        # [batch_sz, enc_units]
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights, coverage


class Decoder(tf.keras.layers.Layer):
    """
    calculate output before pointer generator network
    input dec_inp [batch_sz, 1], hidden [batch_sz, enc_units], context_vector [batch_sz, enc_units]
    output dec_inp_context [batch_sz,1,embedding_dim+enc_units] dec_pred [batch_size,vocab_size] dec_hidden [batch_sz,dec_units]
    """
    def __init__(self, dec_units, batch_sz, embedding_matrix):
        super(Decoder, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape[0], embedding_matrix.shape[1]
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size, activation=tf.keras.activations.softmax)

    def call(self, dec_inp, dec_hidden, context_vector):
        # context_vector[batch_sz, enc_units]
        # dec_hidden [batch_sz, dec_units] NOTE dec_units == enc_units
        # dec_inp shape [batch_size, 1, embedding_dim]
        dec_inp = self.embedding(dec_inp)
        # dec_inp shape [batch_sz, 1, embedding_dim + enc_units]
        dec_inp_context = tf.concat([tf.expand_dims(context_vector, 1), dec_inp], axis=-1)
        # output [batch_sz, 1, dec_units] state [batch_sz, dec_units]
        output, dec_hidden = self.gru(dec_inp_context)
        # output shape [batch_size, dec_units]
        output = tf.reshape(output, (-1, output.shape[2]))
        # print('output deduced by dec_hidden', tf.math.reduce_sum(output-dec_hidden)) they are same!!!
        # dec_inp shape [batch_size, vocab_size]
        dec_pred = self.fc(output)
        return dec_inp_context, dec_pred, dec_hidden


class Pointer(tf.keras.layers.Layer):
    """
    calculate Pgen
    input context_vector [batch_sz,enc_units] dec_hidden [batch_sz,dec_units] dec_inp_context [batch_sz,1,embedding_dim+enc_units]
    output scaler pgen
    """
    def __init__(self):
        super(Pointer, self).__init__()
        self.w_s_reduce = tf.keras.layers.Dense(1)
        self.w_i_reduce = tf.keras.layers.Dense(1)
        self.w_c_reduce = tf.keras.layers.Dense(1)

    def call(self, context_vector, dec_hidden, dec_inp):
        # change dec_inp_context to [batch_sz,embedding_dim+enc_units]
        dec_inp = tf.squeeze(dec_inp, axis=1)
        pgen = tf.nn.sigmoid(self.w_s_reduce(dec_hidden) + self.w_c_reduce(context_vector) + self.w_i_reduce(dec_inp))
        return pgen

class PGN(tf.keras.Model):
    """
    create pgn model
    input
    output
    """
    def __init__(self, gru_units, att_units, batch_sz, embedding_matrix):
        super(PGN, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape[0], embedding_matrix.shape[1]
        self.enc_units = gru_units
        self.dec_units = gru_units
        self.att_units = att_units
        self.encoder = Encoder(self.enc_units, batch_sz, embedding_matrix)
        self.attention = BahdanauAttention(self.att_units)
        self.decoder = Decoder(self.dec_units, batch_sz, embedding_matrix)
        self.pointer = Pointer()

    def call_encoder(self, enc_inp):
        enc_hidden = self.encoder.initialize_hidden_state()    # [batch_sz, enc_units]
        enc_output, enc_hidden = self.encoder(enc_inp, enc_hidden)   # [batch_sz, max_train_x, enc_units], [batch_sz, enc_units]
        return enc_output, enc_hidden

    # NOTE dec_inp [batch_sz, max_train_y]
    def call(self, dec_inp, enc_extended_inp, enc_pad_mask, batch_oov_len, enc_output, dec_hidden,
             use_coverage=True, prev_coverage=None, prediction=False):
        predictions = []
        attentions = []
        coverages = []
        p_gens = []

        # # initiate_hidden and context_vector for pgn
        # enc_hidden = self.encoder.initialize_hidden_state()
        # enc_output, enc_hidden = self.encoder(enc_inp, enc_hidden)  # same as above
        # # initiate coverage to None or whatever was passed in
        # dec_hidden = enc_hidden
        # # initiate context_vector and coverage_ret

        context_vector, attention_weights, coverage_ret = self.attention(dec_hidden,enc_output,enc_pad_mask,
                                                                         use_coverage,prev_coverage)

        if prediction:
            decode_steps = dec_inp.shape[1]
        else:
            decode_steps = dec_inp.shape[1] - 1

        for t in range(decode_steps):  # 11.11 iterate over time step max_len_y - 1 !!!!

            # attentions.append(attention_weights)
            # decoder takes dec_inp, dec_hidden, context_vector, dec_inp shape [batch_size, 1]
            # dec_hidden [batch_sz, dec_units] NOTE dec_units == enc_units, context_vector[batch_sz, enc_units]
            # decoder gives dec_pred [batch_size,vocab_size] dec_hidden [batch_sz,dec_units]
            # using teaching force!!!
            attentions.append(attention_weights)
            coverages.append(coverage_ret)
            
            dec_inp_context, dec_pred, dec_hidden = self.decoder(tf.expand_dims(dec_inp[:, t], 1), dec_hidden,
                                                                 context_vector)
            # attention takes dec_hidden, enc_output, use_coverage = True, prev_coverage = None
            # attention gives context_vector, attention_weights, coverage
            if not prediction:
                context_vector, attention_weights, coverage_ret = self.attention(dec_hidden, enc_output,
                                                            enc_pad_mask,use_coverage,coverage_ret)
            # pointer takes context_vector, dec_hidden, dec_inp
            pgen = self.pointer(context_vector, dec_hidden, dec_inp_context)
            predictions.append(dec_pred)
            p_gens.append(pgen)


        # enc_extended_input [batch_sz, max_len_x] predictions [max_len_y, batch_sz, vocab_size]
        # attentions [max_len_y, batch_sz, max_len_x, 1] p_gens [max_len_y,]
        final_dist = self._calc_final_dist(enc_extended_inp, predictions, attentions, p_gens, batch_oov_len)
        # change shape of final_dist from [max_len_y, batch_sz, extend_vocab_size]
        # to [batch_sz, max_len_y, extend_vocab_size]
        #final_dist = tf.transpose(final_dist, [1, 0, 2])

        return final_dist, attentions, coverages, dec_hidden, context_vector, p_gens


    def _calc_final_dist(self, enc_extended_inp, vocab_dists, attn_dists, p_gens, batch_oov_len):
        """
        Calculate the final distribution, for the pointer-generator model
        Args:
        vocab_dists, prediction of decoder List length max_dec_steps of (batch_sz, vocab_size) array.
                    The words are in the order they appear in the vocabulary file.
        attn_dists: The attention distributions. List length max_dec_steps of (batch_size, max_train_x, 1) array.
        _enc_batch_extend_vocab, tokenized enc input (batch_sz, max_train_x) with pgn the in-article oov word is
        tonkenized with extended index.
        Returns:
        final_dists: The final distributions. List length max_dec_steps of (batch_size, extended_vocab_size) arrays.
        """
#       enc_extended_inp.shape, vocab_dists[0].shape, attn_dists[0].shape, p_gens[0].shape, batch_oov_len.shape
#       (128, 86)               (128, 9047)           (128, 86, 1)         (128, 1)         (128,)
#         print(len(attn_dists), attn_dists[0].shape)
#         85 (128, 86, 1)

        max_len_y, batch_sz, vocab_size = len(vocab_dists), vocab_dists[0].shape[0], vocab_dists[0].shape[1]
        attn_dists = tf.squeeze(attn_dists, axis = -1) # change to max_dec_steps of (batch_size, max_train_x) array
        batch_oov_len = tf.reduce_max(batch_oov_len)  # the maximum (over the batch) size of the extended vocabulary
        
#         print(attn_dists.shape)
#         (85, 128, 86)
        
        # p_gens = tf.convert_to_tensor(p_gens)
        vocab_dists = [p_gen * dist for (p_gen, dist) in zip(p_gens, vocab_dists)]
        attn_dists = [(1 - p_gen) * dist for (p_gen, dist) in zip(p_gens, attn_dists)]
        # to substitute above code
        # def weight_cross(p_gens, vocab_dists):
        #     list_like = tf.TensorArray(dtype=tf.float32,size=1,dynamic_size=True,clear_after_read=False)
        #     for i in range(len(p_gens)):
        #         list_like = list_like.write(i, p_gens[i] * vocab_dists[i])
        #     return list_like.stack()
        # vocab_dists = weight_cross(p_gens, vocab_dists)
        # attn_dists = weight_cross((1-p_gens), attn_dists)

        # Concatenate some zeros to each vocabulary dist, to hold the probabilities for in-article OOV words
        extended_vocab_size = vocab_size + batch_oov_len
        extra_zeros = tf.zeros((batch_sz, batch_oov_len))
        vocab_dists_extended = [tf.concat(axis=1, values=[dist, extra_zeros]) for dist in vocab_dists]

        # extra_zeros = tf.zeros((max_len_y, batch_sz, batch_oov_len))
        # vocab_dists_extended = tf.concat(axis=2, values=[vocab_dists,extra_zeros])
        # list length max_dec_steps of shape (batch_size, extended_vsize) [max_len_y, batch_sz, extended_vsize]
        # vocab_dists_extended = [tf.concat(axis=1, values=[dist, extra_zeros]) for dist in vocab_dists]
        # to substitute above code


        # Project the values in the attention distributions onto the appropriate entries in the final distributions
        # This means that if a_i = 0.1 and the ith encoder word is w, and w has index 500 in the vocabulary,
        # then we add 0.1 onto the 500th entry of the final distribution
        # This is done for each decoder timestep.
        # This is fiddly; we use tf.scatter_nd to do the projection
        batch_nums = tf.range(0, limit=batch_sz)  # shape (batch_size)
        batch_nums = tf.expand_dims(batch_nums, 1)  # shape (batch_size, 1)
#         print(batch_nums)
        max_len_x = tf.shape(enc_extended_inp)[1]  # number of states we attend over
        batch_nums = tf.tile(batch_nums, [1, max_len_x])  # shape (batch_size, max_train_x)
#         print(batch_nums)
#         [[  0   0   0 ...   0   0   0]
#          [  1   1   1 ...   1   1   1]
#          [  2   2   2 ...   2   2   2]
#          ...
#          [125 125 125 ... 125 125 125]
#          [126 126 126 ... 126 126 126]
#          [127 127 127 ... 127 127 127]]
#         raise
        indices = tf.stack((batch_nums, enc_extended_inp), axis=2)  # shape (batch_size, max_train_x, 2)
#         enc_extended_inp (128, 84)
#         print(indices)
#         [[[   0    2]
#           [   0   24]
#           [   0   55]
#           ...
#           [   0 9045]
#           [   0 9045]
#           [   0 9045]]

#          [[   1    2]
#           [   1 2711]
#           [   1  611]
#           ...
#           [   1 9045]
#           [   1 9045]
#           [   1 9045]]

#          [[ 127    2]
#           [ 127 1187]
#           [ 127    1]
#           ...
#           [ 127 9045]
#           [ 127 9045]
#           [ 127 9045]]], shape=(128, 86, 2)
#         raise
        
        shape = (batch_sz, extended_vocab_size)
        # list length max_dec_steps (batch_size, extended_vocab_size)
        attn_dists_projected = [tf.scatter_nd(indices, copy_dist, shape) for copy_dist in attn_dists]
        # substitute above code
        # temp = tf.TensorArray(dtype=tf.float32,size=1,dynamic_size=True,clear_after_read=False)
        # for i in range(p_gens.shape[0]):  # which equal to max_len_y
        #     temp = temp.write(i, tf.scatter_nd(indices, attn_dists[i], shape))
        # attn_dists_projected = temp.stack()

        # Add the vocab distributions and the copy distributions together to get the final distributions
        # final_dists is a list length max_dec_steps; each entry is a tensor shape (batch_size, extended_vsize) giving
        # the final distribution for that decoder timestep
        # Note that for decoder timesteps and examples corresponding to a [PAD] token, this is junk - ignore.
        final_dists = [vocab_dist+copy_dist for (vocab_dist,copy_dist) in zip(vocab_dists_extended,
                                                                              attn_dists_projected)]
        # to substitute code above
        # final_dists = tf.add(vocab_dists_extended,attn_dists_projected)

        return final_dists

In [3]:
model = PGN(gru_unites, attn_units, batch_size, np.array(inp_vocabulary_matrix))

NameError: name 'gru_unites' is not defined